In [47]:
from keras.models import Sequential , Model
from keras.layers import Dense ,  BatchNormalization , Reshape , Input , Flatten
from keras.layers import Conv2D , MaxPool2D , Conv2DTranspose , UpSampling2D , ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import Activation
from keras.layers import Dropout

from keras.layers import Embedding

from keras.layers import Multiply

from keras.optimizers import Adam

from keras.initializers import truncated_normal , random_normal , constant

In [42]:
import os

import matplotlib as plt
import numpy as np

import pandas as pd

import gc

%matplotlib inline

In [15]:
#CIFAR10 dataset
WIDTH = 32
HEIGHT = 32
CHANNEL = 3

LATENT_DIM = 100 #latent variable z sample from normal distribution

BATCH_SIZE = 64
EPOCHS = 10

PATH = '../dataset/train/'

#生成多少个图像 长*宽
ROW = 5
COL = 5

#=========
#=========
#add new FLAG(s)
CLASS_NUM = 10 #mnist=10 CIFAR10=10 CIFAR100=100 CIFAR1000=1000

LABEL2INDEX = {'frog':0 , 'truck':1 , 'deer':2 , 'automobile':3 , 'bird':4 , 'horse':5 , 'ship':6 , 'cat':7 , 'dog':8 , 'airplane':9}
INDEX2LABEL = {value:key for key , value in LABEL2INDEX.items()}

In [16]:
INDEX2LABEL

{0: 'frog',
 1: 'truck',
 2: 'deer',
 3: 'automobile',
 4: 'bird',
 5: 'horse',
 6: 'ship',
 7: 'cat',
 8: 'dog',
 9: 'airplane'}

In [39]:

load_index = 0

images_name = os.listdir(PATH)

IMAGES_COUNT = len(images_name)


In [27]:
train_labels = pd.read_csv('../dataset/trainLabels.csv')
#每张图片对应的类别标号
train_labels = train_labels['label'].map(LABEL2INDEX).get_values()

In [5]:
'''
(X_train , y_train),(X_test , y_test) = mnist.load_data()
X_train = X_train/127.5-1
X_train = np.expand_dims(X_train , 3)
y_train = np.expand_dims(y_train , 1)
'''

In [55]:
'''
def load_mnist():
    random_index = np.random.randint(0, X_train.shape[0], BATCH_SIZE)
    return X_train[random_index] , y_train[random_index]
    
def write_image_mnist(epoch):
    
    noise = np.random.normal(size = (ROW*COL , LATENT_DIM))
    label = np.random.randint(low=0 , high=CLASS_NUM , size=(ROW*COL , 1)) #CLASS_NUM 是否需要加1
    
    generated_image = generator_i.predict([noise , label])
    generated_image = generated_image*0.5+0.5
    
    fig , axes = plt.pyplot.subplots(ROW , COL)
    fig.subplots_adjust(hspace=0.9 , wspace=0.9) #将每一个图像在高度 宽度 上进行缩放或者放大
    #fig.tight_layout()
    
    count=0
    
    for i in range(ROW):
        for j in range(COL):
            axes[i][j].imshow(generated_image[count,:,:,0] , cmap = 'gray')
            axes[i][j].axis('off')
            axes[i][j].set_title('%d' % (label[i*ROW+j]))
            count += 1
    
    
    fig.savefig('mnist_acgan/No.%d.png' % (epoch))
    plt.pyplot.close()
'''

In [67]:

def load_image(batch_size = BATCH_SIZE):
    global load_index
    
    images = []
    labels = []
    
    for i in range(batch_size):
        images.append(plt.image.imread(PATH + images_name[(load_index + i) % IMAGES_COUNT]))
        labels.append(train_labels[(load_index + i) % IMAGES_COUNT])
    
    load_index += batch_size
    
    return np.array(images)/127.5-1 , np.array(labels)

def write_image(epoch):
    
    noise = np.random.normal(size = (ROW*COL , LATENT_DIM))
    labels = np.random.randint(low=0 , high=CLASS_NUM , size=(ROW*COL , 1))
    
    generated_image = generator_i.predict([noise , labels])
    
    generated_image = (generated_image+1)*127.5
    
    fig , axes = plt.pyplot.subplots(ROW , COL)
    fig.subplots_adjust(hspace=0.9 , wspace=0.9)
    
    count = 0
    
    for i in range(ROW):
        for j in range(COL):
            axes[i][j].imshow(generated_image[count])
            axes[i][j].axis('off')
            axes[i][j].set_title(INDEX2LABEL[labels[i*ROW+j][0]])
            
            count += 1
            
    fig.savefig('CIFAR10_acgan/No.%d.png' % epoch)
    plt.pyplot.close()
    
    #plt.image.imsave('images/'+str(epoch)+'.jpg')


In [44]:
def conv2d(output_size):
    return Conv2D(output_size , kernel_size=(5,5) , strides=(2,2) , padding='same' , kernel_initializer=truncated_normal(stddev=0.02) , bias_initializer=constant(0.0))

def dense(output_size):
    return Dense(output_size , kernel_initializer=random_normal(stddev=0.02) , bias_initializer=constant(0.0))

def deconv2d(output_size):
    return Conv2DTranspose(output_size , kernel_size=(5,5) , strides=(2,2) , padding='same' , kernel_initializer=random_normal(stddev=0.02) , bias_initializer=constant(0.0))

def batch_norm():
    return BatchNormalization(momentum=0.9 , epsilon=1e-5)


In [54]:
def generator():
    #sample from noise z
    model = Sequential(name='generator')
    
    #CIFAR10 图像使用 32*32*3
    model.add(Dense(2 * 2 * 64*8, activation="relu", input_shape=(LATENT_DIM,)))
    model.add(Reshape((2, 2, 64*8)))
    
    model.add(batch_norm())
    model.add(LeakyReLU(alpha=0.2))
    #4
    model.add(deconv2d(64*4))
    model.add(batch_norm())
    model.add(LeakyReLU(alpha=0.2))
    #8
    model.add(deconv2d(64*2))
    model.add(batch_norm())
    model.add(LeakyReLU(alpha=0.2))
    #16
    model.add(deconv2d(64*1))
    model.add(batch_norm())
    model.add(LeakyReLU(alpha=0.2))
    #32
    model.add(deconv2d(3))
    model.add(Activation('tanh'))
    
    #model.summary()
    
    noise = Input(shape=(LATENT_DIM , ) , name='input1')
    label = Input(shape=(1,) , dtype='int32')
    
    _ = Embedding(input_dim=CLASS_NUM , output_dim=LATENT_DIM)(label)
    embedding_label = Flatten()(_)
    
    noise_embedding_label = Multiply()([noise , embedding_label]) #(None , LATENT_DIM)
    
    image = model(noise_embedding_label)
    
    return Model([noise , label] , image , name='generator_Model')

In [55]:
def discriminator():
    #input a image to discriminate real or fake
    model = Sequential(name='discriminator')
    
    model.add(Conv2D(filters=64 , kernel_size=(5,5) , strides=(2,2) , padding='same' , input_shape=(WIDTH , HEIGHT , CHANNEL) , kernel_initializer=truncated_normal(stddev=0.02) , bias_initializer=constant(0.0) , name='conv1'))
    model.add(LeakyReLU(alpha=0.2))
    
    model.add(conv2d(64*2))
    model.add(batch_norm())
    model.add(LeakyReLU(alpha=0.2))
    
    model.add(conv2d(64*4))
    model.add(batch_norm())
    model.add(LeakyReLU(alpha=0.2))

    model.add(conv2d(64*8))
    model.add(batch_norm())
    model.add(LeakyReLU(alpha=0.2))
    
    model.add(Flatten())
    
    #model.summary()
    
    image = Input(shape=(WIDTH , HEIGHT , CHANNEL) , name='input1')
    flatten_feature = model(image)
    
    #FC层 多加了一层
    _ = Dense(128)(flatten_feature)
    _ = LeakyReLU(alpha=0.2)(_)
    
    validity = Dense(1 , activation='sigmoid')(_)
    
    label_hat = Dense(CLASS_NUM , activation='softmax')(flatten_feature) #CLASS_NUM位置是否需要加1 双输出 双损失函数 为每一个输出指定损失权重
    
    return Model(image , [validity , label_hat] , name='discriminator_Model')

In [52]:
def combined_model(generator_i , discriminator_i):
    #生成器和判别器组合成整体
    z = Input(shape=(LATENT_DIM , ) , name='z')
    label = Input(shape=(1,) , dtype='int32')
    
    image = generator_i([z , label])
    
    #print(image.shape)
    
    discriminator_i.trainable = False
    validity , label_hat = discriminator_i(image)
    
    return Model([z , label] , [validity , label_hat] , name='combined_model')

In [53]:
adam = Adam(lr = 0.0002 , beta_1=0.5)

In [56]:
discriminator_i = discriminator()
discriminator_i.compile(optimizer=adam , loss=['binary_crossentropy' , 'sparse_categorical_crossentropy'] , metrics=['accuracy'])

generator_i = generator()

combined_model_i = combined_model(generator_i , discriminator_i)


combined_model_i.compile(optimizer=adam , loss=['binary_crossentropy' , 'sparse_categorical_crossentropy'])

In [ ]:
real_labels = np.ones(shape=(BATCH_SIZE , 1)) #真实样本label为1
fake_labels = np.zeros(shape=(BATCH_SIZE , 1)) #假样本label为0

for i in range(10000):
    for j in range(int(IMAGES_COUNT/BATCH_SIZE)):
        
        noise = np.random.normal(size=(BATCH_SIZE , LATENT_DIM))
        corresponding_fake_label = np.random.randint(low=0 , high=CLASS_NUM , size=(BATCH_SIZE , 1)) #label的取值范围 可能会发生变化

        real_image , corresponding_real_label = load_image()

        #训练判别器
        fake_image = generator_i.predict([noise , corresponding_fake_label])

        real_loss = discriminator_i.train_on_batch(real_image , [real_labels , corresponding_real_label])
        fake_loss = discriminator_i.train_on_batch(fake_image , [fake_labels , corresponding_fake_label])

        loss = np.add(real_loss , fake_loss)/2

        #训练生成器
        noise2 = np.random.normal(size=(BATCH_SIZE , LATENT_DIM))
        corresponding_fake_label2 = np.random.randint(low=0 , high=CLASS_NUM , size=(BATCH_SIZE , 1))

            #下面的损失是一个list 有两个损失 一个是validity一个是与label的softmax
        generator_loss = combined_model_i.train_on_batch([noise2 , corresponding_fake_label2] , [real_labels , corresponding_fake_label2])

        print('epoch:%d batch:%d loss:%f accu:%f gene_loss:[validity:%f softmax:%f]' % (i , j , loss[0] , loss[1] , generator_loss[0] , generator_loss[1]))

        if i % 100 == 0:
            write_image(i)
    
write_image(999)


C:\Anaconda3\lib\site-packages\keras\engine\training.py:478: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


epoch:0 loss:3.096836 accu:0.949856 gene_loss:[validity:2.852037 softmax:1.055460]
epoch:1 loss:2.894385 accu:0.777178 gene_loss:[validity:3.034246 softmax:1.264032]
epoch:2 loss:2.771531 accu:0.721706 gene_loss:[validity:3.180160 softmax:1.473228]
epoch:3 loss:3.037333 accu:0.822526 gene_loss:[validity:3.363082 softmax:1.354850]
epoch:4 loss:3.115052 accu:0.930309 gene_loss:[validity:3.660562 softmax:1.569029]
epoch:5 loss:3.415852 accu:1.234845 gene_loss:[validity:2.989946 softmax:1.003390]
epoch:6 loss:2.954998 accu:0.833495 gene_loss:[validity:2.986782 softmax:1.174665]
epoch:7 loss:2.937807 accu:0.796385 gene_loss:[validity:3.046650 softmax:1.264905]
epoch:8 loss:2.930703 accu:0.771459 gene_loss:[validity:3.091355 softmax:1.294886]
epoch:9 loss:3.038326 accu:0.847192 gene_loss:[validity:3.309506 softmax:1.411303]
epoch:10 loss:3.080025 accu:0.897966 gene_loss:[validity:3.535561 softmax:1.564080]
epoch:11 loss:3.441851 accu:1.239967 gene_loss:[validity:3.050072 softmax:0.981274]
ep

epoch:99 loss:2.964922 accu:0.843641 gene_loss:[validity:2.877574 softmax:1.104244]
epoch:100 loss:3.094653 accu:0.840135 gene_loss:[validity:2.892363 softmax:1.110088]
epoch:101 loss:2.851655 accu:0.833203 gene_loss:[validity:2.702723 softmax:0.985800]
epoch:102 loss:2.901866 accu:0.797696 gene_loss:[validity:2.884081 softmax:1.130809]
epoch:103 loss:2.971777 accu:0.854552 gene_loss:[validity:2.823543 softmax:1.080214]
epoch:104 loss:2.915628 accu:0.879953 gene_loss:[validity:2.815588 softmax:1.014390]
epoch:105 loss:2.855647 accu:0.800327 gene_loss:[validity:2.677713 softmax:1.024090]
epoch:106 loss:2.954493 accu:0.832635 gene_loss:[validity:2.812590 softmax:0.962923]
epoch:107 loss:3.023187 accu:0.789126 gene_loss:[validity:2.739357 softmax:1.003788]
epoch:108 loss:2.742124 accu:0.755928 gene_loss:[validity:2.741678 softmax:0.955318]
epoch:109 loss:2.886312 accu:0.794922 gene_loss:[validity:2.960398 softmax:1.148796]
epoch:110 loss:3.010190 accu:0.878356 gene_loss:[validity:3.176930

epoch:197 loss:2.944552 accu:0.843368 gene_loss:[validity:2.471264 softmax:1.001754]
epoch:198 loss:2.712959 accu:0.773342 gene_loss:[validity:2.409337 softmax:1.190451]
epoch:199 loss:2.636683 accu:0.752428 gene_loss:[validity:2.590742 softmax:1.106456]
epoch:200 loss:3.026723 accu:0.823623 gene_loss:[validity:2.581478 softmax:0.942089]
epoch:201 loss:2.925982 accu:0.860777 gene_loss:[validity:2.457435 softmax:1.056660]
epoch:202 loss:2.917724 accu:0.830563 gene_loss:[validity:2.781578 softmax:1.125053]
epoch:203 loss:2.946614 accu:0.831914 gene_loss:[validity:2.521434 softmax:1.002594]
epoch:204 loss:2.793469 accu:0.774437 gene_loss:[validity:2.527017 softmax:1.073299]
epoch:205 loss:2.701300 accu:0.777871 gene_loss:[validity:2.477798 softmax:1.020336]
epoch:206 loss:2.695042 accu:0.787375 gene_loss:[validity:2.415352 softmax:1.057683]
epoch:207 loss:2.806770 accu:0.752714 gene_loss:[validity:2.744781 softmax:1.260472]
epoch:208 loss:2.750907 accu:0.820637 gene_loss:[validity:2.40775

epoch:294 loss:2.716575 accu:0.773449 gene_loss:[validity:2.456502 softmax:0.990198]
epoch:295 loss:2.645553 accu:0.766589 gene_loss:[validity:2.660460 softmax:1.353313]
epoch:296 loss:2.698103 accu:0.788775 gene_loss:[validity:2.178393 softmax:1.059346]
epoch:297 loss:2.629843 accu:0.727757 gene_loss:[validity:2.457077 softmax:1.172652]
epoch:298 loss:2.739311 accu:0.804157 gene_loss:[validity:2.275118 softmax:1.013088]
epoch:299 loss:2.526285 accu:0.738569 gene_loss:[validity:2.470350 softmax:1.241052]
epoch:300 loss:2.654833 accu:0.743045 gene_loss:[validity:2.981493 softmax:1.185859]
epoch:301 loss:3.075387 accu:1.001316 gene_loss:[validity:2.663548 softmax:1.137244]
epoch:302 loss:2.886379 accu:0.910940 gene_loss:[validity:2.511456 softmax:1.094168]
epoch:303 loss:2.780202 accu:0.855880 gene_loss:[validity:2.200957 softmax:1.039481]
epoch:304 loss:2.570884 accu:0.672249 gene_loss:[validity:2.574244 softmax:1.255077]
epoch:305 loss:2.575108 accu:0.744414 gene_loss:[validity:2.27837

...

epoch:2225 loss:2.264824 accu:0.743574 gene_loss:[validity:1.512585 softmax:0.999441]
epoch:2226 loss:2.422512 accu:0.793994 gene_loss:[validity:1.680256 softmax:1.149078]
epoch:2227 loss:2.339570 accu:0.755660 gene_loss:[validity:1.649271 softmax:1.216624]
epoch:2228 loss:2.463976 accu:0.870787 gene_loss:[validity:1.406067 softmax:0.920939]
epoch:2229 loss:2.283051 accu:0.771735 gene_loss:[validity:1.357068 softmax:1.045716]
epoch:2230 loss:2.301902 accu:0.738534 gene_loss:[validity:1.406686 softmax:1.069188]
epoch:2231 loss:2.328022 accu:0.709436 gene_loss:[validity:1.524166 softmax:1.203312]
epoch:2232 loss:2.385486 accu:0.728802 gene_loss:[validity:1.375129 softmax:1.115487]
epoch:2233 loss:2.333857 accu:0.753283 gene_loss:[validity:1.398515 softmax:0.970022]
epoch:2234 loss:2.191630 accu:0.683588 gene_loss:[validity:1.496511 softmax:1.117943]
epoch:2235 loss:2.531363 accu:0.766645 gene_loss:[validity:1.608259 softmax:1.060443]
epoch:2236 loss:2.415642 accu:0.833955 gene_loss:[vali

epoch:2321 loss:2.039815 accu:0.646658 gene_loss:[validity:1.622077 softmax:1.257387]
epoch:2322 loss:2.409646 accu:0.831198 gene_loss:[validity:1.490898 softmax:1.053003]
epoch:2323 loss:2.270894 accu:0.807622 gene_loss:[validity:1.417015 softmax:0.996024]
epoch:2324 loss:2.124753 accu:0.707121 gene_loss:[validity:1.388959 softmax:1.149442]
epoch:2325 loss:2.328003 accu:0.712437 gene_loss:[validity:1.483273 softmax:1.177580]
epoch:2326 loss:2.302416 accu:0.766450 gene_loss:[validity:1.321061 softmax:0.939327]
epoch:2327 loss:2.199573 accu:0.711080 gene_loss:[validity:1.350843 softmax:1.079810]
epoch:2328 loss:2.107184 accu:0.693556 gene_loss:[validity:1.427826 softmax:0.981547]
epoch:2329 loss:2.110351 accu:0.684926 gene_loss:[validity:1.474405 softmax:1.179433]
epoch:2330 loss:2.299679 accu:0.719686 gene_loss:[validity:1.437774 softmax:1.069855]
epoch:2331 loss:2.260728 accu:0.749918 gene_loss:[validity:1.642667 softmax:0.985891]
epoch:2332 loss:2.409821 accu:0.795275 gene_loss:[vali

epoch:2417 loss:2.188323 accu:0.700662 gene_loss:[validity:1.333179 softmax:1.028169]
epoch:2418 loss:2.269781 accu:0.725824 gene_loss:[validity:1.380489 softmax:1.075960]
epoch:2419 loss:2.416938 accu:0.763694 gene_loss:[validity:1.370094 softmax:1.013861]
epoch:2420 loss:2.338560 accu:0.770179 gene_loss:[validity:1.358019 softmax:0.997810]
epoch:2421 loss:2.438934 accu:0.759022 gene_loss:[validity:1.498168 softmax:0.977702]
epoch:2422 loss:2.271801 accu:0.720398 gene_loss:[validity:1.371637 softmax:1.110455]
epoch:2423 loss:2.099241 accu:0.682109 gene_loss:[validity:1.319912 softmax:1.143201]
epoch:2424 loss:2.251338 accu:0.738492 gene_loss:[validity:1.305422 softmax:0.983834]
epoch:2425 loss:2.211993 accu:0.715788 gene_loss:[validity:1.406559 softmax:1.057114]
epoch:2426 loss:2.339641 accu:0.767895 gene_loss:[validity:1.347516 softmax:1.030116]
epoch:2427 loss:2.516415 accu:0.802467 gene_loss:[validity:1.256747 softmax:0.847368]
epoch:2428 loss:2.231616 accu:0.694981 gene_loss:[vali

epoch:2513 loss:2.167413 accu:0.659866 gene_loss:[validity:1.332008 softmax:1.085604]
epoch:2514 loss:2.354402 accu:0.759837 gene_loss:[validity:1.363948 softmax:0.975535]
epoch:2515 loss:2.459887 accu:0.773172 gene_loss:[validity:1.340897 softmax:0.891821]
epoch:2516 loss:2.155958 accu:0.726109 gene_loss:[validity:1.357435 softmax:1.104432]
epoch:2517 loss:2.152699 accu:0.717667 gene_loss:[validity:1.285858 softmax:1.064399]
epoch:2518 loss:2.367057 accu:0.785473 gene_loss:[validity:1.307230 softmax:1.018152]
epoch:2519 loss:2.403666 accu:0.773072 gene_loss:[validity:1.496786 softmax:0.943188]
epoch:2520 loss:2.160321 accu:0.764200 gene_loss:[validity:1.275499 softmax:1.058239]
epoch:2521 loss:2.302633 accu:0.694408 gene_loss:[validity:1.205753 softmax:1.025104]
epoch:2522 loss:2.266559 accu:0.743919 gene_loss:[validity:1.228222 softmax:0.921443]
epoch:2523 loss:2.148201 accu:0.679309 gene_loss:[validity:1.400020 softmax:1.126975]
epoch:2524 loss:2.198047 accu:0.776309 gene_loss:[vali

epoch:2609 loss:2.171413 accu:0.730524 gene_loss:[validity:1.244525 softmax:0.969971]
epoch:2610 loss:2.238204 accu:0.708555 gene_loss:[validity:1.229190 softmax:0.980330]
epoch:2611 loss:2.205148 accu:0.711404 gene_loss:[validity:1.285191 softmax:0.955918]
epoch:2612 loss:2.301629 accu:0.715675 gene_loss:[validity:1.200332 softmax:1.026633]
epoch:2613 loss:2.150947 accu:0.695019 gene_loss:[validity:1.283527 softmax:1.031560]
epoch:2614 loss:2.127460 accu:0.712835 gene_loss:[validity:1.229940 softmax:0.980296]
epoch:2615 loss:2.182615 accu:0.717646 gene_loss:[validity:1.280143 softmax:1.013466]
epoch:2616 loss:2.108688 accu:0.731412 gene_loss:[validity:1.271239 softmax:0.985445]
epoch:2617 loss:2.237719 accu:0.743171 gene_loss:[validity:1.208219 softmax:0.990662]
epoch:2618 loss:2.220909 accu:0.733126 gene_loss:[validity:1.283904 softmax:0.994715]
epoch:2619 loss:2.483290 accu:0.795388 gene_loss:[validity:1.299413 softmax:0.937966]
epoch:2620 loss:2.307432 accu:0.749723 gene_loss:[vali

In [ ]:
for i in range(100000):
    noise = np.random.normal(size=(BATCH_SIZE , LATENT_DIM))
    corresponding_fake_label = np.random.randint(low=0 , high=CLASS_NUM , size=(BATCH_SIZE , 1)) #label的取值范围 可能会发生变化

    real_image , corresponding_real_label = load_image()

    #训练判别器
    fake_image = generator_i.predict([noise , corresponding_fake_label])

    real_loss = discriminator_i.train_on_batch(real_image , [real_labels , corresponding_real_label])
    fake_loss = discriminator_i.train_on_batch(fake_image , [fake_labels , corresponding_fake_label])

    loss = np.add(real_loss , fake_loss)/2

    #训练生成器
    noise2 = np.random.normal(size=(BATCH_SIZE , LATENT_DIM))
    corresponding_fake_label2 = np.random.randint(low=0 , high=CLASS_NUM , size=(BATCH_SIZE , 1))

        #下面的损失是一个list 有两个损失 一个是validity一个是与label的softmax
    generator_loss = combined_model_i.train_on_batch([noise2 , corresponding_fake_label2] , [real_labels , corresponding_fake_label2])

    print('epoch:%d loss:%f accu:%f gene_loss:[validity:%f softmax:%f]' % (i , loss[0] , loss[1] , generator_loss[0] , generator_loss[1]))

    if i % 100 == 0:
        write_image(i+10000)
    
write_image(999)


In [70]:
real_labels.shape

(64, 1)

In [53]:
discriminator_i.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input1 (InputLayer)          (None, 28, 28, 1)         0         
_________________________________________________________________
discriminator (Sequential)   (None, 1)                 533505    
Total params: 533,505
Trainable params: 0
Non-trainable params: 533,505
_________________________________________________________________


In [54]:
generator_i.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input1 (InputLayer)          (None, 100)               0         
_________________________________________________________________
generator (Sequential)       (None, 28, 28, 1)         1097744   
Total params: 1,097,744
Trainable params: 1,095,184
Non-trainable params: 2,560
_________________________________________________________________


In [39]:
combined_model_i.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
z (InputLayer)               (None, 100)               0         
_________________________________________________________________
generator_Model (Model)      (None, 96, 96, 3)         29029120  
_________________________________________________________________
discriminator_Model (Model)  (None, 1)                 14320641  
Total params: 43,349,761
Trainable params: 29,025,536
Non-trainable params: 14,324,225
_________________________________________________________________


In [ ]:
gc.collect()

In [14]:
modeli = Sequential()

modeli.add(Dense(128 * 7 * 7, activation="relu", input_shape=(LATENT_DIM,)))
modeli.add(Reshape((7, 7, 128)))
modeli.add(UpSampling2D())
modeli.add(Conv2D(128, kernel_size=3, padding="same"))
modeli.add(BatchNormalization(momentum=0.8))
modeli.add(Activation("relu"))
modeli.add(UpSampling2D())
modeli.add(Conv2D(64, kernel_size=3, padding="same"))
modeli.add(BatchNormalization(momentum=0.8))
modeli.add(Activation("relu"))
modeli.add(Conv2D(CHANNEL, kernel_size=3, padding="same"))
modeli.add(Activation("tanh"))

In [15]:
modeli.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 6272)              633472    
_________________________________________________________________
reshape_3 (Reshape)          (None, 7, 7, 128)         0         
_________________________________________________________________
up_sampling2d_2 (UpSampling2 (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 128)       147584    
_________________________________________________________________
batch_normalization_2 (Batch (None, 14, 14, 128)       512       
_________________________________________________________________
activation_1 (Activation)    (None, 14, 14, 128)       0         
_________________________________________________________________
up_sampling2d_3 (UpSampling2 (None, 28, 28, 128)       0         
__________

In [41]:
32*400

12800

In [ ]:
gc.collect()